# Imports

In [22]:
import pandas as pd

# import py files
from Classes import Player, Team, LeagueIndividual
from GA_mutation import mutation_swap_players

# Local Data

In [23]:
df = pd.read_csv("data/players.csv")
df = df.drop(columns=['Unnamed: 0']) # drop the index column

# Problem Configuration

In [24]:
TEAM_SIZE = 7
NUM_TEAMS = 5
BUDGET_LIMIT = 750
TEAM_STRUCTURE = {"GK": 1, "DEF": 2, "MID": 2, "FWD": 2}
POPULATION_SIZE = 10

# Convert DF to player objects

In [25]:
players_by_position = {
    pos: [Player.from_dict(row) for _, row in df[df['Position'] == pos].iterrows()]
    for pos in TEAM_STRUCTURE
}

# Generate Population

In [26]:
# === GENERATE POPULATION ===
def generate_initial_population(size, players_by_position, team_structure, budget_limit, num_teams):
    population = []
    attempts = 0
    max_attempts = 1000 # avoid infinite loop if unable to generate valid leagues

    while len(population) < size and attempts < max_attempts:
        indiv = LeagueIndividual(players_by_position, team_structure, budget_limit, num_teams)
        if indiv.league is not None:
            population.append(indiv)
        attempts += 1

    return population

population = generate_initial_population(
    POPULATION_SIZE,
    players_by_position,
    TEAM_STRUCTURE,
    BUDGET_LIMIT, 
    NUM_TEAMS
)

# Check if Classes are working

In [27]:
# === EXAMPLE USAGE ===
individual = LeagueIndividual(players_by_position, TEAM_STRUCTURE, BUDGET_LIMIT, NUM_TEAMS)

# Print result
print("\n=== One League Example ===")
for i, team in enumerate(individual.league):
    print(f"\n🏆 Team {i + 1}")
    print(team)
    print(f"Avg Skill: {team.avg_skill():.2f} | Total Salary: €{team.total_salary()}M")

print(f"\nFitness: {individual.fitness:.4f}")


=== One League Example ===

🏆 Team 1
  - GK: Alex Carter | Skill: 85 | Salary: €90M
  - DEF: Maxwell Flores | Skill: 81 | Salary: €72M
  - DEF: Daniel Foster | Skill: 90 | Salary: €110M
  - MID: Ashton Phillips | Skill: 90 | Salary: €110M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - FWD: Sebastian Perry | Skill: 95 | Salary: €150M
  - FWD: Landon Powell | Skill: 89 | Salary: €110M
Avg Skill: 88.00 | Total Salary: €737M

🏆 Team 2
  - GK: Jordan Smith | Skill: 88 | Salary: €100M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - DEF: Jaxon Griffin | Skill: 79 | Salary: €65M
  - MID: Nathan Wright | Skill: 92 | Salary: €120M
  - MID: Gavin Richardson | Skill: 87 | Salary: €95M
  - FWD: Xavier Bryant | Skill: 90 | Salary: €120M
  - FWD: Tyler Jenkins | Skill: 80 | Salary: €70M
Avg Skill: 85.71 | Total Salary: €655M

🏆 Team 3
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - DEF: Brayden Hughes | Skill: 87 | Salary: €100M
  - M

In [28]:


# === PRINT POPULATION DETAILS ===
for idx, indiv in enumerate(population):
    print("\n" + "=" * 35)
    print(f"🏟️  League (Individual) {idx + 1}")
    print("=" * 35)

    for tidx, team in enumerate(indiv.league):
        print(f"\n  🏆 Team {tidx + 1}")
        print(team)
        print(f"    📊 Avg Skill: {team.avg_skill():.2f}")
        print(f"    💰 Total Salary: €{team.total_salary()}M")

    print(f"\n  ➤ League Fitness (Std Dev of team avg skill): {indiv.fitness:.4f}")



🏟️  League (Individual) 1

  🏆 Team 1
  - GK: Jordan Smith | Skill: 88 | Salary: €100M
  - DEF: Maxwell Flores | Skill: 81 | Salary: €72M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - MID: Dylan Morgan | Skill: 91 | Salary: €115M
  - MID: Ashton Phillips | Skill: 90 | Salary: €110M
  - FWD: Colton Gray | Skill: 91 | Salary: €125M
  - FWD: Landon Powell | Skill: 89 | Salary: €110M
    📊 Avg Skill: 87.71
    💰 Total Salary: €717M

  🏆 Team 2
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Ethan Howard | Skill: 80 | Salary: €70M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - MID: Gavin Richardson | Skill: 87 | Salary: €95M
  - MID: Spencer Ward | Skill: 84 | Salary: €85M
  - FWD: Sebastian Perry | Skill: 95 | Salary: €150M
  - FWD: Adrian Collins | Skill: 85 | Salary: €90M
    📊 Avg Skill: 85.29
    💰 Total Salary: €665M

  🏆 Team 3
  - GK: Ryan Mitchell | Skill: 83 | Salary: €85M
  - DEF: Jaxon Griffin | Skill: 79 | Salary: €65M
  - DEF: Daniel Foster | Skill: 90

# Mutations

## Swap Players Betweeen Teams

In [29]:
swap_players = mutation_swap_players(individual)
print(swap_players)

<LeagueIndividual fitness=0.6540>


In [30]:
if swap_players == individual:
    print("⚠️  No mutation applied (swap failed after multiple attempts).")
else:
    print("success")

success


In [32]:
# Print result
print("\n=== MUTATED (mutation_swap_players()) ===")
for i, team in enumerate(swap_players.league):
    print(f"\n🏆 Team {i + 1}")
    print(team)
    print(f"Avg Skill: {team.avg_skill():.2f} | Total Salary: €{team.total_salary()}M")

print(f"\nFitness: {swap_players.fitness:.4f}")


=== MUTATED (mutation_swap_players()) ===

🏆 Team 1
  - GK: Chris Thompson | Skill: 80 | Salary: €80M
  - DEF: Maxwell Flores | Skill: 81 | Salary: €72M
  - DEF: Daniel Foster | Skill: 90 | Salary: €110M
  - MID: Ashton Phillips | Skill: 90 | Salary: €110M
  - MID: Dominic Bell | Skill: 86 | Salary: €95M
  - FWD: Sebastian Perry | Skill: 95 | Salary: €150M
  - FWD: Landon Powell | Skill: 89 | Salary: €110M
Avg Skill: 87.29 | Total Salary: €727M

🏆 Team 2
  - GK: Blake Henderson | Skill: 87 | Salary: €95M
  - DEF: Caleb Fisher | Skill: 84 | Salary: €85M
  - DEF: Jaxon Griffin | Skill: 79 | Salary: €65M
  - MID: Nathan Wright | Skill: 92 | Salary: €120M
  - MID: Connor Hayes | Skill: 89 | Salary: €105M
  - FWD: Xavier Bryant | Skill: 90 | Salary: €120M
  - FWD: Tyler Jenkins | Skill: 80 | Salary: €70M
Avg Skill: 85.86 | Total Salary: €660M

🏆 Team 3
  - GK: Alex Carter | Skill: 85 | Salary: €90M
  - DEF: Logan Brooks | Skill: 86 | Salary: €95M
  - DEF: Brayden Hughes | Skill: 87 | Salar